In [ ]:
#default_exp pde

In [ ]:
#exporti
import torch
import warnings
import numpy as np
from scipy.sparse import diags, csc_matrix

from dl4to.utils import get_σ_vm
from dl4to.pde import SparseLinearSolver, PDESolver, FDMDerivatives, FDMAdjointDerivatives, FDMAssembly, UnpaddedFDM

In [ ]:
#hide
from nbdev.showdoc import show_doc

# FDM solver

In [ ]:
#export
class FDM(UnpaddedFDM):
    """
    A PDE solver for linear elasticity that uses the finite differences method (FDM) with padding.
    """
    def __init__(self, θ_min:float=1e-6, # The minimal value in the stiffness matrix. For numerical reasons we can not allow 0s, since they may lead to singular matrices.
                 use_forward_differences:bool=True, # Whether to use forward differences or central differences.
                 assemble_tensors_when_passed_to_problem:bool=True, # Whether the PDE solver methods pre-assembles any tensors or arrays before solving the PDE for a concrete problem.
                 padding_depth:int=0 # The depth of the padding surrounding the design space. In some cases, it is recommended to increase the padding depth to 2 to improve results but also increase running time.
                ):
        self.padding_depth = padding_depth
        super().__init__(
            θ_min=θ_min,
            use_forward_differences=use_forward_differences,
            assemble_tensors_when_passed_to_problem=assemble_tensors_when_passed_to_problem
        )


    @property
    def shape(self):
        return self.Ω_dirichlet.shape[-3:]


    @property
    def Ω_dirichlet(self):
        return self._get_padded_tensor(self.problem.Ω_dirichlet)


    def _get_padded_tensor(self, tensor):
        p_d = int(self.padding_depth)
        if p_d == 0:
            return tensor

        shape = tensor.shape
        assert len(shape) == 4
        padded_tensor = torch.zeros(
            shape[0], shape[1]+2*p_d, shape[2]+2*p_d, shape[3]+2*p_d, dtype=tensor.dtype
        )
        padded_tensor[:, p_d:-p_d, p_d:-p_d, p_d:-p_d] = tensor
        return padded_tensor


    def _remove_padding(self, tensor):
        p_d = int(self.padding_depth)
        if p_d == 0:
            return tensor

        shape = tensor.shape
        assert len(shape) == 4
        return tensor[:, p_d:-p_d, p_d:-p_d, p_d:-p_d]


    def _get_θ_from_solution(self, solution, binary=False, clone=False):
        θ = super()._get_θ_from_solution(solution, binary=binary, clone=clone)
        return self._get_padded_tensor(θ)


    def _A(self, u, θ, dirichlet=True, p=1.):
        if θ.shape[-3:] == self.shape[-3:]:
            return super()._A(u=u, θ=θ, dirichlet=dirichlet, p=p)
        θ_padded = self._get_padded_tensor(θ)
        assert θ_padded.shape[-3:] == self.shape[-3:]
        return super()._A(u=u, θ=θ_padded, dirichlet=dirichlet, p=p)


    def _A_adj(self, y, θ, dirichlet=True, p=1.):
        if θ.shape[-3:] == self.shape[-3:]:
            return super()._A_adj(y=y, θ=θ, dirichlet=dirichlet, p=p)
        θ_padded = self._get_padded_tensor(θ)
        assert θ_padded.shape[-3:] == self.shape[-3:]
        return super()._A_adj(y=y, θ=θ_padded, dirichlet=dirichlet, p=p)


    def _get_b(self):
        b = self.problem.F
        b = self._get_padded_tensor(b)
        b[self.Ω_dirichlet] = 0
        b /= self.problem.E
        return b


    def _get_u(self, solution, p=1., binary=False, get_padded=False):
        u = super()._get_u(solution, p=p, binary=binary)
        if get_padded:
            return u
        return self._remove_padding(u)


    def _get_σ(self, solution, p=1., u=None, binary=False, get_padded=False):
        if u is None:
            u = self._get_u(solution, p=p, binary=binary, get_padded=True)
        σ = super()._get_σ(solution, p=p, u=u, binary=binary)
        if get_padded:
            return σ
        return self._remove_padding(σ)


    def solve_pde(self, 
                 solution:"dl4to.solution.Solution", # The solution for which the PDE should be solved.
                 p:float=1., # The SIMP exponent when solving the PDE. Should usually be left at its default value of `1.`.
                 binary:bool=False, # Whether the densities in the solution should be binarized before solving the PDE.
                 get_padded:bool=False # Whether the density should be padded before the PDE is solved. Takes a bit longer to solve, but is more accurate.
                ):
        """
        Solves the pde for `solution` and SIMP exponent `p`. Returns three `torch.Tensor` objects: displacements `u`, stresses `σ` and von Mises stresses `σ_vm`.
        """
        u = self._get_u(solution, p=p, binary=binary, get_padded=True)
        σ = self._get_σ(solution, p=p, u=u, binary=binary, get_padded=True)
        σ_vm = get_σ_vm(σ)
        if get_padded:
            return u, σ, σ_vm
        return self._remove_padding(u), self._remove_padding(σ), self._remove_padding(σ_vm)

NameError: name 'UnpaddedFDM' is not defined

In [ ]:
show_doc(FDM.assemble_tensors)

<h4 id="UnpaddedFDM.assemble_tensors" class="doc_header"><code>UnpaddedFDM.assemble_tensors</code><a href="dl4to/pde.py#L639" class="source_link" style="float:right">[source]</a></h4>

> <code>UnpaddedFDM.assemble_tensors</code>(**`problem`**)

Assembles all FDM tensors from the problem object that do not require a density θ.

In [ ]:
show_doc(FDM.solve_pde)

<h4 id="FDM.solve_pde" class="doc_header"><code>FDM.solve_pde</code><a href="__main__.py#L98" class="source_link" style="float:right">[source]</a></h4>

> <code>FDM.solve_pde</code>(**`solution`**:`dl4to.solution.Solution`, **`p`**:`float`=*`1.0`*, **`binary`**:`bool`=*`False`*, **`get_padded`**:`bool`=*`True`*)

Solves the pde for `solution` and SIMP exponent `p`. Returns three `torch.Tensor` objects: displacements `u`, stresses `σ` and von Mises stresses `σ_vm`.

||Type|Default|Details|
|---|---|---|---|
|**`solution`**|`dl4to.solution.Solution`||The solution for which the PDE should be solved.|
|**`p`**|`float`|`1.0`|The SIMP exponent when solving the PDE. Should usually be left at its default value of `1.`.|
|**`binary`**|`bool`|`False`|Whether the densities in the solution should be binarized before solving the PDE.|
|**`get_padded`**|`bool`|`True`|Whether the density should be padded before the PDE is solved. Takes a bit longer to solve, but is more accurate.|


In [ ]:
#hide
from dl4to.solution import Solution
from dl4to.datasets import BasicDataset

In [ ]:
#hide
atol = 1e-3
dtype = torch.float64

In [ ]:
#hide
def get_rand_θ(problem):
    θ = torch.rand(1,*problem.shape).clamp_(problem.pde_solver.θ_min, 1)
    θ = θ**5

    θ[problem.Ω_design == 0] = 0
    θ[problem.Ω_design == 1] = 1
    return θ


def get_mock_objects(force=-1.5e5, resolution=35, padding_depth=2):
    """Mock Objects with Padding Depth 2"""
    problem = BasicDataset(resolution=resolution, dtype=dtype).ledge(force_per_area=force)
    problem.pde_solver = FDM(padding_depth=padding_depth)
    θ = get_rand_θ(problem)
    solution = Solution(problem, θ, enforce_θ_on_Ω_design=False)

    shape_prod = np.prod(problem.shape)
    u = torch.randn(3, *problem.shape, dtype=dtype)

    return problem, problem.pde_solver, θ, solution, shape_prod, u

In [ ]:
%%time
#hide

def test_that_A_op_and_A_mat_sum_coincide():
    problem, fdm, θ, solution, shape_prod, u = get_mock_objects(padding_depth=2)
    u = torch.rand(3, *fdm.shape, dtype=dtype) / 1e5
#     θ = torch.rand(1, *fdm.shape, dtype=dtype)

    Au_op = fdm._A(u, θ, dirichlet=True)
    Au_mat = torch.tensor(fdm._assemble_A(fdm._get_padded_tensor(θ)).dot(u.flatten()))
    torch.allclose(Au_op.flatten().abs().sum(), Au_mat.flatten().abs().sum(), rtol=0), Au_op.flatten().abs().sum()


test_that_A_op_and_A_mat_sum_coincide()

CPU times: user 16.6 s, sys: 108 ms, total: 16.7 s
Wall time: 3.34 s


In [ ]:
%%time
#slow
#hide

def test_that_u_solves_linear_system():
    problem, fdm, θ, solution, shape_prod, u = get_mock_objects(padding_depth=2)

    u = fdm.solve_pde(solution, get_padded=True)[0]
    Au = fdm._A(u, θ)
    assert Au.shape == fdm.b.shape
    assert torch.allclose(Au, fdm.b, atol=1e-2, rtol=0), torch.norm(Au - fdm.b)

test_that_u_solves_linear_system()

CPU times: user 1min 36s, sys: 257 ms, total: 1min 37s
Wall time: 13.5 s


In [ ]:
%%time
#slow
#hide

def test_that_u_solves_linear_system_in_norm():
    problem, fdm, θ, solution, shape_prod, u = get_mock_objects(padding_depth=2)

    u = fdm.solve_pde(solution, get_padded=True)[0]
    norm_error = (fdm._A(u, θ) - fdm.b).norm()
    norm_b = fdm.b.norm()
    assert norm_error / norm_b < 1e-1, norm_error / norm_b

test_that_u_solves_linear_system_in_norm()

CPU times: user 1min 41s, sys: 229 ms, total: 1min 41s
Wall time: 14.8 s


In [ ]:
%%time
#broken
#hide

def test_that_u_grows_linearly_with_applied_forces():
    force = -1.5e5
    n = torch.randint(0, 100, (1,))
    problem, fdm, θ, solution, shape_prod, _ = get_mock_objects(force=force, padding_depth=0)
    problem2, fdm2, θ2, solution2, shape_prod2, _ = get_mock_objects(force=force * n, padding_depth=0)
    solution2 = Solution(problem2, θ)
    u = fdm.solve_pde(solution, get_padded=False)[0]
    u2 = fdm2.solve_pde(solution2, get_padded=False)[0]

    assert torch.allclose(n * u, u2)

test_that_u_grows_linearly_with_applied_forces()

CPU times: user 20.9 s, sys: 67.9 ms, total: 21 s
Wall time: 3.36 s


In [ ]:
%%time
#hide

def test_if_A_and_At_are_adjoint():
    problem, fdm, θ, solution, shape_prod, u = get_mock_objects(padding_depth=2)

    u = torch.rand(3, *fdm.shape, dtype=dtype) / 1e5
    v = torch.rand(3, *fdm.shape, dtype=dtype) / 1e5
    Au = fdm._A(u, θ , dirichlet=True)
    Atv = fdm._A_adj(v, θ, dirichlet=True)
    assert torch.allclose(torch.dot(u.flatten(), Atv.flatten()), torch.dot(Au.flatten(), v.flatten()), atol=1e-1, rtol=0)

test_if_A_and_At_are_adjoint()

CPU times: user 16 s, sys: 67.8 ms, total: 16.1 s
Wall time: 3.05 s


In [ ]:
%%time
#hide

def test_that_padded_0_is_equal_to_unpadded_fdm_solver():
    problem, fdm_padded0, θ, solution, shape_prod, u = get_mock_objects(padding_depth=0)
    solution_clone = solution.clone()
    fdm_unpadded = UnpaddedFDM()
    solution_clone.problem.pde_solver = fdm_unpadded

    u_padded0 = solution.solve_pde()[0]
    u_unpadded = solution_clone.solve_pde()[0]


    assert torch.allclose(u_padded0, u_unpadded)

test_that_padded_0_is_equal_to_unpadded_fdm_solver()

CPU times: user 21.9 s, sys: 43.4 ms, total: 22 s
Wall time: 3.41 s


In [ ]:
%%time
#hide

def test_tensile_rod_solution_is_close_to_theoretical_case(tol=0.001):
    force_per_area = -1.5e5
    tensile_rod_problem = BasicDataset(resolution=30).tensile_rod(force_per_area=force_per_area)
    tensile_rod_problem.pde_solver = FDM(use_forward_differences=True, padding_depth=2)
    solution = tensile_rod_problem.trivial_solution
    u_pde, σ, _ = solution.solve_pde()

    relative_error = force_per_area / -σ[-1, 1, 1, 10].item()
    assert 1. - tol < relative_error < 1. + tol, relative_error


test_tensile_rod_solution_is_close_to_theoretical_case()

CPU times: user 23.4 s, sys: 67.3 ms, total: 23.4 s
Wall time: 3.27 s


In [ ]:
%%time
#slow
#hide

def test_that_ledge_solution_is_close_to_theoretical_beam_displacement(tol=0.25):
    force_per_area = -1.5e5
    ledge_problem = BasicDataset(resolution=30).ledge(force_per_area=force_per_area)
    ledge_problem.pde_solver = FDM(use_forward_differences=True, padding_depth=0)
    solution = ledge_problem.trivial_solution
    u_pde, _, _ = solution.solve_pde()
    pde_max_displacement = u_pde.min().item()

    #theoretical solution
    l, b, h = ledge_problem.size[:]
    I = (b*h**3)/12
    force_per_length = force_per_area * b
    E = ledge_problem.E
    theoretical_max_displacement = force_per_length*(l**4) /(8*E*I)

    relative_error = abs(theoretical_max_displacement / pde_max_displacement)
    assert 1. - tol < relative_error < 1. + tol, relative_error

test_that_ledge_solution_is_close_to_theoretical_beam_displacement()

CPU times: user 4.83 s, sys: 16.5 ms, total: 4.84 s
Wall time: 862 ms


In [ ]:
%%time
#slow
#hide

def test_displacements_dont_propagate_through_void__check_forks_displacements_in_the_top_are_at_least_5times_greater_than_in_the_bottom():
    fork_problem = BasicDataset(resolution=35).fork()
    fork_problem.pde_solver = FDM(padding_depth=0)
    solution = fork_problem.trivial_solution
    solution.θ *= 1e-6

    u = solution.solve_pde()[0]

    displacements_in_the_top = u[2, -1, :, -1].mean().abs()
    displacements_in_the_bottom = u[2, -1, :, 0].mean().abs()

    assert displacements_in_the_top > 5 * displacements_in_the_bottom, displacements_in_the_top / displacements_in_the_bottom

test_displacements_dont_propagate_through_void__check_forks_displacements_in_the_top_are_at_least_5times_greater_than_in_the_bottom()

CPU times: user 10.9 s, sys: 4.15 ms, total: 10.9 s
Wall time: 1.64 s
